In [1]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dash import Dash, dcc, html, Input, Output, State

In [3]:
def harmonic_with_noise(amplitude, frequency, phase, noise_mean, noise_covariance, show_noise, time):
    harmonic = amplitude * np.sin(2 * np.pi * frequency * time + phase)
    noise = np.random.normal(noise_mean, np.sqrt(noise_covariance), size=len(time))
    if show_noise:
        return harmonic + noise, noise
    return harmonic, None

def custom_filter(signal, window_size):
    if window_size < 1:
        raise ValueError("Розмір вікна має бути >= 1")

    kernel = np.ones(window_size) / window_size
    filtered_signal = np.convolve(signal, kernel, mode='same')
    return filtered_signal

initial_params = {
    "amplitude": 1.0,
    "frequency": 1.0,
    "phase": 0.0,
    "noise_mean": 0.0,
    "noise_covariance": 0.1,
    "show_noise": True,
    "filter_window_size": 5
}

time = np.linspace(0, 2, 1000)

app = Dash(__name__)

app.layout = html.Div([
    html.H1("Гармоніка із шумом"),

    dcc.Graph(id="harmonic-plot"),

    html.Div([
        html.Div([
            html.Label(id="amplitude-label", children=f"Амплітуда: {initial_params['amplitude']:.1f}"),
            dcc.Slider(id="amplitude-slider", min=0.1, max=5.0, step=0.1, value=initial_params["amplitude"]),
        ], style={"margin-bottom": "20px"}),

        html.Div([
            html.Label(id="frequency-label", children=f"Частота: {initial_params['frequency']:.1f}"),
            dcc.Slider(id="frequency-slider", min=0.1, max=5.0, step=0.1, value=initial_params["frequency"]),
        ], style={"margin-bottom": "20px"}),

        html.Div([
            html.Label(id="phase-label", children=f"Фазовий зсув: {initial_params['phase']:.1f}"),
            dcc.Slider(id="phase-slider", min=0.0, max=2 * np.pi, step=0.1, value=initial_params["phase"]),
        ], style={"margin-bottom": "20px"}),

        html.Div([
            html.Label(id="noise-mean-label", children=f"Середнє шуму: {initial_params['noise_mean']:.1f}"),
            dcc.Slider(id="noise-mean-slider", min=-1.0, max=1.0, step=0.1, value=initial_params["noise_mean"]),
        ], style={"margin-bottom": "20px"}),

        html.Div([
            html.Label(id="noise-covariance-label", children=f"Дисперсія шуму: {initial_params['noise_covariance']:.2f}"),
            dcc.Slider(id="noise-covariance-slider", min=0.01, max=1.0, step=0.01, value=initial_params["noise_covariance"]),
        ], style={"margin-bottom": "20px"}),

        html.Div([
            html.Label(id="filter-window-label", children=f"Розмір вікна фільтру: {initial_params['filter_window_size']}"),
            dcc.Slider(id="filter-window-slider", min=1, max=50, step=1, value=initial_params["filter_window_size"]),
        ], style={"margin-bottom": "20px"}),

        html.Label("Відображення шуму"),
        dcc.Checklist(id="show-noise-checkbox", options=[{"label": "Шум", "value": "show_noise"}],
                      value=["show_noise"] if initial_params["show_noise"] else []),

        html.Label("Вибір графіка"),
        dcc.Dropdown(
            id="plot-dropdown",
            options=[
                {"label": "Тільки гармоніка", "value": "harmonic"},
                {"label": "Тільки шум", "value": "noise"},
                {"label": "Обидва", "value": "both"},
                {"label": "Фільтрована гармоніка", "value": "filtered"},
            ],
            value="both",
        ),

        html.Button("Reset", id="reset-button", n_clicks=0),
    ], style={"width": "50%", "margin": "auto"})
])

@app.callback(
    Output("harmonic-plot", "figure"),
    Input("amplitude-slider", "value"),
    Input("frequency-slider", "value"),
    Input("phase-slider", "value"),
    Input("noise-mean-slider", "value"),
    Input("noise-covariance-slider", "value"),
    Input("filter-window-slider", "value"),
    Input("show-noise-checkbox", "value"),
    Input("plot-dropdown", "value"),
    State("harmonic-plot", "figure")
)
def update_plot(amplitude, frequency, phase, noise_mean, noise_covariance, filter_window, show_noise, plot_selection, current_figure):
    show_noise_flag = "show_noise" in show_noise
    harmonic, noise = harmonic_with_noise(amplitude, frequency, phase, noise_mean, noise_covariance, show_noise_flag, time)

    if plot_selection == "filtered":
        harmonic = custom_filter(harmonic, filter_window)

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Гармоніка", "Шум"))

    if plot_selection in ["harmonic", "both", "filtered"]:
        fig.add_trace(go.Scatter(x=time, y=harmonic, mode="lines", name="Гармоніка"), row=1, col=1)

    if plot_selection in ["noise", "both"] and noise is not None:
        fig.add_trace(go.Scatter(x=time, y=noise, mode="lines", name="Шум", line=dict(dash="dot")), row=2, col=1)

    fig.update_layout(height=600, title="Гармоніка із шумом")

    return fig

@app.callback(
    [Output("amplitude-label", "children"),
     Output("frequency-label", "children"),
     Output("phase-label", "children"),
     Output("noise-mean-label", "children"),
     Output("noise-covariance-label", "children"),
     Output("filter-window-label", "children")],
    [Input("amplitude-slider", "value"),
     Input("frequency-slider", "value"),
     Input("phase-slider", "value"),
     Input("noise-mean-slider", "value"),
     Input("noise-covariance-slider", "value"),
     Input("filter-window-slider", "value")]
)
def update_labels(amplitude, frequency, phase, noise_mean, noise_covariance, filter_window):
    return (
        f"Амплітуда: {amplitude:.1f}",
        f"Частота: {frequency:.1f}",
        f"Фазовий зсув: {phase:.1f}",
        f"Середнє шуму: {noise_mean:.1f}",
        f"Дисперсія шуму: {noise_covariance:.2f}",
        f"Розмір вікна фільтру: {filter_window}"
    )

@app.callback(
    [Output("amplitude-slider", "value"),
     Output("frequency-slider", "value"),
     Output("phase-slider", "value"),
     Output("noise-mean-slider", "value"),
     Output("noise-covariance-slider", "value"),
     Output("filter-window-slider", "value"),
     Output("show-noise-checkbox", "value"),
     Output("plot-dropdown", "value")],
    Input("reset-button", "n_clicks")
)
def reset_parameters(n_clicks):
    return (
        initial_params["amplitude"],
        initial_params["frequency"],
        initial_params["phase"],
        initial_params["noise_mean"],
        initial_params["noise_covariance"],
        initial_params["filter_window_size"],
        ["show_noise"] if initial_params["show_noise"] else [],
        "both",
    )

app.run_server(debug=True)
